<a href="https://colab.research.google.com/github/kwafafoa/neqsimpython/blob/master/AaHTEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install neqsim

In [65]:
import neqsim
from neqsim.thermo.thermoTools import*
import pandas as pd


In [80]:
inputdata = {
    'feed1Temperature':26.00,
    'feed1Pressure': 87.20,
    'feed1Flowrate': 588200.00,
    'cooler1T': 21.80,
    'heater1T': 27.00,
    'heater1P': 86.2,
    'leanTEGFlowRate':200.00,
    'leanTEGTemperature': 35.3,
    'numberOfEquilibriumStagesTEGabsorber': 3,
  'stageEfficiencyTEGabsorber': 1.0,



    
}

In [69]:
from neqsim.thermo import fluid, printFrame
feedFluid1 = fluid('cpa')
feedFluid1 = {'ComponentName':['water', 'TEG', 'nitrogen', 'CO2','methane','ethane','propane','i-butane','n-butane','22-Mpropane','i-pentane','n-pentane','HaklangC6*', 'HaklangC7*','HaklangC8*','HaklangC9*','HaklangC10*'],
               'MolarComposition[-]':[0.000423,0.000,0.008229,0.005630,0.929444,0.034861,0.012438,0.002266,0.003084,0.000038,0.001048,0.000870,0.000799,0.000822,0.000040,0.000007,0.000001],
               'MolarMass[kg/mol]': [None,None,None,None,None,None,None,None,None,None,None,None,0.08420,0.0905,0.10320,0.11750,0.13400],
               'RelativeDensity[-]': [None,None,None,None,None,None,None,None,None,None,None,None,0.66980,0.73940,0.76900,0.78740,0.79580]
}

Inletstream = pd.DataFrame(feedFluid1)
print(Inletstream.head(20).to_string())
fluid1 = fluid_df(Inletstream)

feedTEG = fluid1.clone()
feedTEG.setMolarComposition([0.05,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])




   ComponentName  MolarComposition[-]  MolarMass[kg/mol]  RelativeDensity[-]
0          water             0.000423                NaN                 NaN
1            TEG             0.000000                NaN                 NaN
2       nitrogen             0.008229                NaN                 NaN
3            CO2             0.005630                NaN                 NaN
4        methane             0.929444                NaN                 NaN
5         ethane             0.034861                NaN                 NaN
6        propane             0.012438                NaN                 NaN
7       i-butane             0.002266                NaN                 NaN
8       n-butane             0.003084                NaN                 NaN
9    22-Mpropane             0.000038                NaN                 NaN
10     i-pentane             0.001048                NaN                 NaN
11     n-pentane             0.000870                NaN                 NaN

In [87]:
from neqsim.process import clearProcess, runProcess,stream,cooler,separator,heater,simpleTEGAbsorber
clearProcess()

feedStream1 = stream(fluid1)
feedStream1.setTemperature(inputdata['feed1Temperature'],'C')
feedStream1.setPressure(inputdata['feed1Pressure'],'barg')
feedStream1.setFlowRate(inputdata['feed1Flowrate'],'kg/hr')

cooler1 = cooler(feedStream1)
cooler1.setOutTemperature(inputdata['cooler1T'],'C')

Scrubber1 = separator(cooler1.getOutStream())
gasFromScrubber1= stream(Scrubber1.getGasOutStream())

heater1 = heater(Scrubber1.getGasOutStream())
heater1.setOutTemperature(inputdata['heater1T'],'C')
heater1.setOutPressure(inputdata['heater1P'],'barg')

feedToabsorber = stream(heater1.getOutStream())

TEGFeed = stream(feedTEG)
TEGFeed.setFlowRate(inputdata['leanTEGFlowRate'], 'kg/hr')
TEGFeed.setTemperature(inputdata['leanTEGTemperature'], 'C')
TEGFeed.setPressure(inputdata['heater1P'], 'barg')

absorber = simpleTEGAbsorber()
absorber.addGasInStream(feedToabsorber)
absorber.addSolventInStream(TEGFeed)
absorber.setNumberOfStages(inputdata['numberOfEquilibriumStagesTEGabsorber'])
absorber.setStageEfficiency(inputdata['stageEfficiencyTEGabsorber'])
absorber.setInternalDiameter(1.7)


dehydratedGas = stream(absorber.getGasOutStream())
richTEG = stream(absorber.getSolventOutStream())






runProcess()

In [89]:
printFrame(richTEG.getFluid())

                         total               gas           aqueous                  
             water  7.83383E-1        2.72562E-4        7.83557E-1   [mole fraction]
               TEG  2.15818E-1       2.99092E-11        2.15866E-1   [mole fraction]
          nitrogen  2.56863E-6        5.90795E-3        1.25727E-6   [mole fraction]
               CO2  1.14066E-4        6.00453E-3        1.12758E-4   [mole fraction]
           methane  6.65942E-4        9.50576E-1        4.55003E-4   [mole fraction]
            ethane  1.44078E-5        3.28824E-2        7.10911E-6   [mole fraction]
           propane   6.3403E-7        2.64564E-3        4.66749E-8   [mole fraction]
          i-butane   1.3531E-8        6.05389E-5       9.06101E-11   [mole fraction]
          n-butane   2.8674E-8         1.2781E-4       2.98723E-10   [mole fraction]
         i-pentane  1.48455E-9        6.67872E-6       1.78957E-12   [mole fraction]
         n-pentane  1.15298E-9        5.18767E-6       1.24945E-1